In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)

learning_rate = 0.1
num_epochs = 15

idx2char = ['h', 'i', 'e', 'l', 'o']

x_data = [[0], [1], [0], [2], [3], [3]]
x_data = Variable(torch.Tensor(x_data))

y_data = [1, 0, 2, 3, 3, 4]

x_one_hot = torch.zeros(6, 5)
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.Tensor(y_data)).long()

num_classes = 5
input_size = 5 # one-hot의 크기, feature의 개수와 같은 의미.
hidden_size = 5
batch_size = 1 # 한 문장, x_one_hot.size(0)과 같은 의미이다.
sequence_length = 6 # 결과의 길이
num_layers = 1

class RNN(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        # RNN의 batch_first parameter는 default로 False이다.
        # 만약 True라면 input과 output이 (batch_size, sequence_length, input_dim)이 된다.
        # False면 input과 output이 (sequence_length, batch_size, input_dim)이 된다.
        # RNN의 activation function은 기본이 tanh이고, ReLU로 바꿀 수 있다.
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # h_0은 rnn의 두 번째 input 인자로, initial hidden state의 값을 나타낸다.
        # h_0은 (num_layers * num_directions, batch_size, hidden_size)크기의 행렬이다.
        # 그런데 batch_first가 True이기 때문에 (batch_size, num_layers * num_directions, hidden_size)크기의 행렬이다.
        h_0 = Variable(torch.zeros(x.size(0), self.num_layers, self.hidden_size))
        # x는 rnn의 첫 번째 input 인자로, (seq_len, batch_size, input_size)크기의 행렬이다.
        # 그런데 rnn 선언시 batch_first가 True라서 (batch_size, seq_len, input_size이다.)
        x.view(x.size(0), self.sequence_length, self.input_size)
        
        # _ 자리는 h_n으로 계산 후의 hidden state 값이다.
        out, _ = self.rnn(x, h_0)
        out = out.view(-1, self.hidden_size)
        out = self.fc(out)
        return out
    
rnn = RNN(num_classes, input_size, hidden_size, num_layers)


cost_func  = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # 아래처럼 대입하는 걸보니 rnn의 forward 메서드가 자동 호출이 되나 보다.
    Y_hat = rnn(inputs)
    cost = cost_func(Y_hat, labels)
    cost.backward()
    optimizer.step()
    _, idx = Y_hat.max(1)
    idx = idx.data.numpy()
    # squeeze 메서드는 행렬 크기 중에 모든 1을 없애 버린다.
    # (A, 1, B, 1) -> (A, B)로 바꾼다는 말
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, cost.data[0]))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

epoch: 1, loss: 1.645
Predicted string:  oooooo
epoch: 2, loss: 1.448
Predicted string:  llllll
epoch: 3, loss: 1.308
Predicted string:  llllll
epoch: 4, loss: 1.144
Predicted string:  illlll
epoch: 5, loss: 0.954
Predicted string:  ilello
epoch: 6, loss: 0.770
Predicted string:  ihello
epoch: 7, loss: 0.612
Predicted string:  ihello
epoch: 8, loss: 0.486
Predicted string:  ihello
epoch: 9, loss: 0.374
Predicted string:  ihello
epoch: 10, loss: 0.272
Predicted string:  ihello
epoch: 11, loss: 0.199
Predicted string:  ihello
epoch: 12, loss: 0.147
Predicted string:  ihello
epoch: 13, loss: 0.105
Predicted string:  ihello
epoch: 14, loss: 0.078
Predicted string:  ihello
epoch: 15, loss: 0.055
Predicted string:  ihello
Learning finished!


d:\program files\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:85: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
